In [1]:
import numpy as np
import pandas as pd
import sqlite3
import dask.dataframe as ddf
import matplotlib.pyplot as plt
import seaborn as sns

# plt.style.use("classic")
# %matplotlib inline
sns.set_style('darkgrid')

In [2]:
db = sqlite3.connect('../input/california-traffic-collision-data-from-switrs/switrs.sqlite')
cur = db.cursor()
tables = [name[0] for name in cur.execute("SELECT name FROM sqlite_master")]
tables = ['collisions', 'parties']

In [3]:
cols = {
    "collisions":
    [
        "case_id"
        
        # Involved
        ,"injured_victims"
        ,"party_count"
        # Collision Types
        ,'pedestrian_collision'
        ,'bicycle_collision'
        ,'motorcycle_collision'
        ,'truck_collision'
        ,'tow_away'
        # Location
#         ,"county_location"
        ,"population"
        ,"intersection"
        ,"state_highway_indicator"
        # Time
        ,"collision_date"
        ,"collision_time"
        # Condition
        ,"weather_1"
        ,"lighting"
        # Violation
        ,"alcohol_involved"
        ,"pcf_violation_category"
        # Condition
        ,"road_surface"
#         ,"road_condition_1"
        ,"location_type"
        ,"type_of_collision"
        ,"control_device"
    ],
    
    "parties":
    [
        "case_id"
        ,"party_number"
        # Who is usually at fault
        ,"at_fault"
        # Demographics
        ,"party_age"
        ,"party_sex"
        # Drugs or Alcohol
        ,"party_drug_physical"
        # Bonus
        ,"party_type"
        ,"party_safety_equipment_1"
        ,"party_safety_equipment_2"
        ,"cellphone_in_use"
    ],
}

def get_str_list(list):
    string = ""
    for col in list:
        string += col + ", "
    string = string[:-2]
    return string

df = {}
date_threshold = "2016-01-01"

In [4]:
# cmd = \
#     "SELECT c.case_id\
#         ,"+get_str_list(cols["collisions"][1:])+"\
#         ,p.drivers_using_phone\
#         ,p.drivers_on_drugs\
#         ,p.parties_not_safe\
#     FROM collisions c\
#     LEFT JOIN\
#         (SELECT case_id\
#                 ,SUM(CASE WHEN party_type='driver' THEN cellphone_in_use END) AS drivers_using_phone\
#                 ,SUM(CASE WHEN party_type='driver' AND party_drug_physical='under drug influence' THEN 1\
#                             ELSE 0\
#                         END) AS drivers_on_drugs\
#                 ,SUM(CASE WHEN party_safety_equipment_1 IN ('air bag not deployed',\
#                         'none in vehicle','lap/shoulder harness not used','passive restraint not used',\
#                         'no child restraint in vehicle','lap belt not used','driver, motorcycle helmet not used',\
#                         'shoulder harness not used','passenger, motorcycle helmet not used','child restraint in vehicle not used',\
#                         'child restraint in vehicle, improper use') \
#                     THEN 1\
#                     WHEN party_safety_equipment_1 IN ('unknown','not required','other','child restraint in vehicle, use unknown')\
#                     THEN 0.5\
#                     ELSE 0\
#                     END) AS parties_not_safe\
#         FROM parties\
#         GROUP BY case_id) p\
#     ON c.case_id = p.case_id\
#     WHERE c.collision_date>='{}'".format(date_threshold)

In [5]:
# Read collisions table
cmd = \
    "SELECT c.case_id\
        ,"+get_str_list(cols["collisions"][1:])+"\
        ,p.drivers_using_phone\
        ,p.drivers_on_drugs\
        ,p.parties_not_safe\
    FROM collisions c\
    LEFT JOIN\
        (SELECT case_id\
                ,SUM(CASE WHEN party_type='driver' THEN cellphone_in_use END) AS drivers_using_phone\
                ,SUM(CASE WHEN party_type='driver' AND party_drug_physical='under drug influence' THEN 1\
                            ELSE 0\
                        END) AS drivers_on_drugs\
                ,SUM(CASE WHEN party_safety_equipment_1 IN ('air bag deployed') OR\
                            party_safety_equipment_2 IN ('air bag deployed')\
                    THEN 0\
                    WHEN party_safety_equipment_1 IN ('lap/shoulder harness used','lap belt used','shoulder harness used') OR\
                            party_safety_equipment_2 IN ('lap/shoulder harness used','lap belt used','shoulder harness used')\
                    THEN 0.5\
                    ELSE 1\
                    END) AS parties_not_safe\
        FROM parties\
        GROUP BY case_id) p\
    ON c.case_id = p.case_id\
    WHERE c.collision_date>='{}'".format(date_threshold)
df = pd.DataFrame([list(x) for x in cur.execute(cmd)], columns=cols["collisions"]+['drivers_using_phone','drivers_on_drugs', 'parties_not_safe'])

# Display
display(df.head())
df.info()

In [6]:
df.groupby('type_of_collision')['case_id'].count().reset_index().sort_values(by="case_id", ascending=False)

In [7]:
# # Read parties table
# cmd = \
#     "SELECT p.case_id\
#         ,"+get_str_list(cols["parties"][1:])+"\
#     FROM parties p\
#     LEFT JOIN \
#         (SELECT case_id, collision_date\
#         FROM collisions) c\
#     ON p.case_id = c.case_id\
#     WHERE collision_date>='{}'".format(date_threshold)
# parties = pd.DataFrame([list(x) for x in cur.execute(cmd)], columns=cols["parties"])

# # Display
# display(parties.head())
# parties.info()

In [8]:
# parties.groupby('party_safety_equipment_2')['case_id'].count().reset_index().sort_values(by='case_id', ascending=False)

# Data Wrangling

In [9]:
display(df.shape)
display(df.isna().sum())

In [10]:
df = df[df['injured_victims'].notna()]
df = df[df['intersection'].notna()]
df = df[df['collision_time'].notna()]
df = df[df['party_count'].notna()]
df = df[df['weather_1'].notna()]
df = df[df['lighting'].notna()]
df = df[df['control_device'].notna()]
df = df[df['type_of_collision'].notna()]
display(df.shape)
display(df.isna().sum())

In [11]:
# Get hour from datetime
def hour(time):
    days, seconds = time.days, time.seconds
    return days * 24 + seconds // 3600

# Adding columns from date
df["collision_date"] = pd.to_datetime(df["collision_date"])
# df["collision_year_month"] = df["collision_date"].dt.to_period('M')
# df["collision_year"] = df["collision_date"].dt.year
# df["collision_month"] = df["collision_date"].dt.month
df['collision_time'] = pd.to_timedelta(df['collision_time'])
df["collision_hour"] = df['collision_time'].apply(lambda x: hour(x))
df["collision_dow"] = df["collision_date"].dt.day_of_week
# df["collision_day_name"] = df["collision_date"].dt.day_name()
# df["collisions"] = df["collisions"][df["collisions"]["collision_year"] == 2020]
df.head()

# EDA

### Evaluating Day of Week

In [12]:
# # Prep df
# df_dow_cases = df.groupby('collision_day_name')['case_id'].nunique().reset_index()
# df_dow_cases_alcohol = df[df["alcohol_involved"]==1].groupby('collision_day_name')['case_id'].nunique().reset_index()
# df_dow_cases = df_dow_cases.rename(columns={'collision_day_name': 'Day of Week', 'case_id': 'Cases'})
# df_dow_cases_alcohol = df_dow_cases_alcohol.rename(columns={'collision_day_name': 'Day of Week', 'case_id': 'Cases with Alcohol'})

# # Visualize
# sns.barplot(x="Day of Week", y="Cases", data=df_dow_cases, palette="hls")
# plt.show()
# sns.barplot(x="Day of Week", y="Cases with Alcohol", data=df_dow_cases_alcohol, palette="hls")
# plt.show()

# # Clear Variable
# del df_dow_cases
# del df_dow_cases_alcohol

 Would be good to separate weekdays and weekends

### Evaluating Time of Day

In [13]:
# # Prep df
# df_tod_cases = df.groupby('collision_hour')['case_id'].nunique().reset_index()
# df_tod_cases = df_tod_cases.rename(columns={'collision_hour': 'Time of Day', 'case_id': 'Cases'})

# # Visualize
# sns.barplot(x="Time of Day", y="Cases", data=df_tod_cases, palette="hls")
# plt.show()

# # Clear Variable
# del df_tod_cases

These hours would have to be grouped together for modeling

In [14]:
# # Prep df
# df_pod_cases = df.groupby('collision_part_of_day')['case_id'].nunique().reset_index()
# df_pod_num_hour = df.groupby('collision_part_of_day')['collision_hour'].nunique().reset_index()
# df_pod_cases = df_pod_cases.merge(df_pod_num_hour, how="inner", on="collision_part_of_day")
# df_pod_cases = df_pod_cases.rename(columns={'collision_part_of_day': 'Part of Day', 'case_id': 'Cases', 'collision_hour': 'Distinct Hours'})
# df_pod_cases['Cases per Hour'] = df_pod_cases['Cases']/df_pod_cases['Distinct Hours']

# df_commute_cases = df.groupby(['is_commute', 'is_weekday'])['case_id'].nunique().reset_index()
# df_commute_cases = df_commute_cases.rename(columns={'is_commute': 'Is Commute', 'is_weekday': 'Is Weekday', 'case_id': 'Cases'})

### Evaluating Months

In [15]:
# # Prep df
# df_month_cases = df.groupby('collision_month')['case_id'].nunique().reset_index()
# df_month_cases = df_month_cases.rename(columns={'collision_month': 'Month', 'case_id': 'Cases'})

# # Visualize
# sns.barplot(x="Month", y="Cases", data=df_month_cases, palette="hls")
# plt.show()

# # Clear variable
# del df_month_cases

No significant differences between months

### Evaluating Lighting

In [16]:
# df.groupby('lighting')['case_id'].nunique()

In [17]:
# # Prep df
# df_lighting_cases = df.groupby('lighting2')['case_id'].nunique().reset_index()
# df_lighting_cases = df_lighting_cases.rename(columns={'lighting2': 'Lighting', 'case_id': 'Cases'})

# # Visualize
# sns.barplot(x="Lighting", y="Cases", data=df_lighting_cases, palette="hls")
# plt.show()

# # Clear variable
# del df_lighting_cases

# Feature Engineering

In [18]:
# Get part of day from collision time
def part_of_day(hour):
    if hour>=23 or hour<1:
        return 'Midnight'
    elif hour>=1 and hour<5:
        return 'Dawn'
    elif hour>=5 and hour<6:
        return 'Early Morning'
    elif hour>=6 and hour<9:
        return 'Morning'
    elif hour>=9 and hour<12:
        return 'Mid-Morning'
    elif hour>=12 and hour<17:
        return 'Afternoon'
    elif hour>=17 and hour<21:
        return 'Evening'
    elif hour>=21 and hour<23:
        return 'Night'

# Check whether collision happened during weekday commute
def is_commute(df):
    hour = df['collision_hour']
    if df['is_weekend']==0:
        if (hour>=5 and hour<=9) or (hour>=17 and hour<=19):
            return 1
        else:
            return 0
    else:
        return 0
        
# Categorize lighting
def lighting_level(lighting):
    dark = ['dark with no street lights','dark with street lights not functioning']
    mid = ['dark with street lights', 'dusk or dawn']
    light = ['daylight']
    
    if lighting in dark:
        return 1
    elif lighting in mid:
        return 0.5
    elif lighting in light:
        return 0
    
# Alcohol Drugs involved
def alcohol_drugs_involved(df):
    alcohol = df['alcohol_involved']
    drugs = df['drivers_on_drugs']
    
    if alcohol==1 or drugs>0:
        return 1
    else:
        return 0
    
# Motorcycle and truck collision
def cycle_truck_collision(df):
    ped = df['pedestrian_collision']
    bike = df['bicycle_collision']
    motor = df['motorcycle_collision']
    truck = df['truck_collision']
    
    if (motor==1 or ped==1 or bike==1) and truck==1:
        return 1
    else:
        return 0

In [19]:
# Label
df['has_injured'] = df['injured_victims'].apply(lambda x: 1 if x>0 else 0)

# Cleaning existing features
# df['alcohol_drugs_involved'] = df['alcohol_involved'].apply(lambda x: 1 if x==1 else 0)
df['alcohol_drugs_involved'] = df[['alcohol_involved','drivers_on_drugs']].apply(alcohol_drugs_involved, axis=1)
df['state_highway_indicator'] = df['state_highway_indicator'].apply(lambda x: 1 if x==1 else 0)
df['tow_away'] = df['tow_away'].apply(lambda x: 1 if x==1 else 0)

# New feature 1:  Is Weekday?
day_type = np.array([0 if x.weekday() < 5 else 1 for x in df['collision_date']])
df['is_weekend'] = day_type

# New feature 2: Part of day
# df['part_of_day'] = df['collision_hour'].apply(lambda x: part_of_day(x))

# New feature 3: Is commute?
df['is_commute'] = df[['is_weekend', 'collision_hour']].apply(is_commute, axis=1)

# New feature 4: Lighting level
df['lighting_level'] = df['lighting'].apply(lambda x: lighting_level(x))

# New feature 5: Road slipperiness road_surface
df['is_slippery'] = df['road_surface'].apply(lambda x: 1 if x in ('wet', 'slippery', 'snowy') else 0)

# New feature 6: Weather wetness weather_1
df['is_wet_weather'] = df['weather_1'].apply(lambda x: 1 if x not in ('clear') else 0)

# New feature 7: Is highway? location_type
df['is_highway'] = df['location_type'].apply(lambda x: 1 if x=='highway' else 0)

# New feature 8: Is broadside? type_of_collision
df['is_broadside'] = df['type_of_collision'].apply(lambda x: 1 if x=='broadside' else 0)
df['is_head-on'] = df['type_of_collision'].apply(lambda x: 1 if x=='head-on' else 0)
df['is_rear_end'] = df['type_of_collision'].apply(lambda x: 1 if x=='rear end' else 0)
df['is_overturned'] = df['type_of_collision'].apply(lambda x: 1 if x=='overturned' else 0)
df['is_pedestrian'] = df['type_of_collision'].apply(lambda x: 1 if x=='pedestrian' else 0)
df['is_broadside_or_headon'] = df['type_of_collision'].apply(lambda x: 1 if x=='broadside' or x=='head-on' else 0)

# New feature 9: Is speeding? pcf_violation_category
df['is_speeding'] = df['pcf_violation_category'].apply(lambda x: 1 if x=='speeding' else 0)
df['is_improper_turn'] = df['pcf_violation_category'].apply(lambda x: 1 if x=='improper turning' else 0)

# New feature 10: Has traffic control device? control_device
df['has_control_device'] = df['control_device'].apply(lambda x: 1 if x=='functioning' else 0)

# New feature 11: Driver using cellphone? drivers_using_phone
df['phone_involved'] = df['drivers_using_phone'].apply(lambda x: 1 if x>0 else 0)

# New feature 12: High population
df['large_population'] = np.array([1 if x == '>250000' else 0 for x in df['population']])

# New feature 13: Motorcycle and truck collision
df['cycle_truck_collision'] = df[['pedestrian_collision','bicycle_collision','motorcycle_collision','truck_collision']].apply(cycle_truck_collision, axis=1)

df.head()

In [20]:
# Existing features
existing_features = ['party_count','parties_not_safe','tow_away'] # , 'intersection'

#, 'is_highway', 'intersection','is_broadside_or_headon','is_broadside','is_head-on','has_control_device','alcohol_drugs_involved','is_slippery','population2','alcohol_drugs_involved','phone_involved','cycle_truck_collision'
new_features = ['is_weekend','is_broadside','is_head-on','is_rear_end','is_overturned','is_pedestrian','is_improper_turn','lighting_level','is_speeding','large_population','is_highway', 'is_slippery', 'is_wet_weather']
features = existing_features + new_features

Xy = df[features+['injured_victims']].dropna()
X = Xy[features]
y = Xy['injured_victims']

display(df.shape)
X.head()

In [21]:
# sns.pairplot(Xy)

In [22]:
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(Xy.corr(), annot=True, cmap='magma', ax=ax)

In [23]:
import statsmodels.api as sm

est = sm.OLS(y,X)
est2 = est.fit()
print(est2.summary())

# Modeling

In [24]:
# Predicting, metrics, etc
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, RidgeCV, LassoCV,ElasticNetCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.utils import resample
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

from xgboost import XGBRegressor

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

When we decide to fit a Linear Regression model, we have to make sure that some conditions are satisfied or else our model will perform poorly or will give us incorrect interpretations.

1. Linearity: X and the mean of Y have a Linear Relationship
2. Homoscedasticity: variance of the error terms is the same for all values of X.
3. No collinearity: independent variables are not highly correlated with each other
4. Normality: Y is normally distributed for any value of X.

If the above four conditions are satisfied, we can expect our Linear Regression model to perform well.

In [26]:
# Linear Regression
model = LinearRegression()
model.fit(X_train, y_train)

# Evaluate performance
y_pred = model.predict(X_test)
print("RMSE:", mean_squared_error(y_test, y_pred, squared=False))
print("R2:", r2_score(y_test, y_pred))
print('Model slope: %0.4f' % model.coef_[0])
print('Model intercept: %0.4f' % model.intercept_)

# Get coefficients
coefficients = pd.concat([pd.DataFrame(X.columns),pd.DataFrame(np.transpose(model.coef_))], axis = 1)
coefficients.columns = ["Feature","Coefficient"]
print('\nFeature importance:')
display(coefficients.sort_values(by="Coefficient", ascending=False))

df_results = pd.DataFrame({'model':'Linear Regression','rmse':mean_squared_error(y_test, y_pred, squared=False), 'r2':r2_score(y_test, y_pred)},  index=[0])

In [27]:
# # Support Vector Machine
# model = SVR(kernel='linear')
# model.fit(X_train, y_train)

# y_pred = model.predict(X_test)
# print("RMSE:", mean_squared_error(y_test, y_pred, squared=False))
# print("R2:", r2_score(y_test, y_pred))

In [28]:
# # ElasticNet Regression
# model = ElasticNet()
# model.fit(X_train, y_train)

# y_pred = model.predict(X_test)
# print("RMSE:", mean_squared_error(y_test, y_pred, squared=False))
# print("R2:", r2_score(y_test, y_pred))
# print('Model slope: %0.4f' % model.coef_[0])
# print('Model intercept: %0.4f' % model.intercept_)

# df_results = df_results.append({'model':'ElasticNet','rmse':mean_squared_error(y_test, y_pred, squared=False), 'r2':r2_score(y_test, y_pred)}, ignore_index=True)

In [29]:
# Random Forest
model = RandomForestRegressor(random_state=1)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("RMSE:", mean_squared_error(y_test, y_pred, squared=False))
print("R2:", r2_score(y_test, y_pred), "\n")

df_results = df_results.append({'model':'Random Forest','rmse':mean_squared_error(y_test, y_pred, squared=False), 'r2':r2_score(y_test, y_pred)}, ignore_index=True)

# Get importance
importance = model.feature_importances_

# Summarize feature importance
cols = list(features)

feature_imp ={}
for i, imp in enumerate(importance):
    feature_imp[cols[i]] = imp
    
sorted_results = {k: v for k, v in sorted(feature_imp.items(), key=lambda item: item[1],reverse=True)}

print("FEATURE : IMPORTANCE")
for x in sorted_results.keys():
    print("{x} : {y}".format(x=x,y=sorted_results[x]))

In [30]:
# XGBoost
model = XGBRegressor(objective='reg:squarederror', n_estimators=1000)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("RMSE:", mean_squared_error(y_test, y_pred, squared=False))
print("R2:", r2_score(y_test, y_pred), "\n")

df_results = df_results.append({'model':'XGBoost','rmse':mean_squared_error(y_test, y_pred, squared=False), 'r2':r2_score(y_test, y_pred)}, ignore_index=True)

# Get importance
importance = model.feature_importances_

# Summarize feature importance
cols = list(features)

feature_imp ={}
for i, imp in enumerate(importance):
    feature_imp[cols[i]] = imp
    
sorted_results = {k: v for k, v in sorted(feature_imp.items(), key=lambda item: item[1],reverse=True)}

print("FEATURE : IMPORTANCE")
for x in sorted_results.keys():
    print("{x} : {y}".format(x=x,y=sorted_results[x]))

In [31]:
df_results